# <font color="red">STEP 1: 文档提取测试</font>

In [ ]:
# 初始化
import django_setup

In [4]:
from apps.projects.models import Project, ProjectStage, DocxExtractionTask

In [ ]:
test_project = Project.objects.get(project_name='测试项目2')
print(f"用于测试的项目: {test_project.project_name}")
print(f"项目包含的文件: {test_project.files.all()}")


In [8]:
# 导入相关模型：get_user_model, Project, FileRecord, DocumentAnalysis, FileProjectLink, ProjectHistory

from apps.projects.services._01_extract_docx_elements import DocxExtractorStep
from apps.projects.services.types import ModelData

In [ ]:
Extractor = DocxExtractorStep()
elements = Extractor.process(ModelData(model=Project, instance=test_project))

In [ ]:
from apps.projects.services.types import ModelData
data = ModelData(model=Project, instance=test_project)
current_project = data.instance
docx_extraction_task = DocxExtractionTask.objects.get(stage__project=current_project)
print(docx_extraction_task.extracted_elements)

In [ ]:
from apps.projects.services.types import DocxElements
docx_elements = DocxElements.from_model(docx_extraction_task.extracted_elements)
print(type(docx_elements))
print(type(docx_elements.elements), type(docx_elements.project))

# <font color="red">STEP 2: outline 分析测试</font>

In [ ]:
# 初始化
import django_setup

In [ ]:
# 定位到测试项目， 和 测试的DocxExtractionTask， 
from apps.projects.models import Project, ProjectStage, DocxExtractionTask
test_project = Project.objects.get(project_name='测试项目2')
print(f"用于测试的项目: {test_project.project_name}")
print(f"项目包含的文件: {test_project.files.all()}")

docx_extraction_task = DocxExtractionTask.objects.get(stage__project=test_project)
print(docx_extraction_task.extracted_elements)


In [ ]:
# 从models.py提取docx_elements
from apps.projects.services.types import DocxElements
docx_elements = DocxElements.from_model(docx_extraction_task.extracted_elements)
print(type(docx_elements))
print(type(docx_elements.elements), type(docx_elements.project))

In [ ]:
from apps.projects.services._02_outline_analysis import DocxOutlineAnalyzerStep

In [ ]:
print(type(docx_elements.project), type(docx_elements.project.instance))
print(docx_elements.project.instance)

In [10]:
outline_analyzer = DocxOutlineAnalyzerStep()


In [ ]:
data_input = outline_analyzer.prepare_requests_data(docx_elements)
print(type(data_input))
from pprint import pprint
pprint(data_input)


In [2]:
# 导入相关模型：get_user_model, Project, FileRecord, DocumentAnalysis, FileProjectLink, ProjectHistory
from django.contrib.auth import get_user_model
from apps.doc_analysis.models import DocumentAnalysis, InvalidStatusTransition
from apps.projects.models import Project
from apps.files.models import FileRecord
from django.core.files.uploadedfile import SimpleUploadedFile
from django.core.exceptions import ValidationError
from apps.doc_analysis.steps._01_extract_docx_elements import DocxExtractorStep
from apps.doc_analysis.pipeline.types import ModelData, DocxElements, OutlineAnalysisResult
from apps.doc_analysis.steps._02_outline_analysis import DocxOutlineAnalyzerStep
from pprint import pprint 

In [ ]:
# 直接引用"测试分析A", 并获取其extracted_elements

saved_analysis = DocumentAnalysis.objects.get(title="测试分析A")
extracted_elements = saved_analysis.extracted_elements
pprint(extracted_elements['elements'][10])

In [4]:
# 创建 DocxElements 实例 from models.py 的 extracted_elements
docx_elements = DocxElements.from_model(extracted_elements)

In [ ]:
print(docx_elements.format_heading_sections())

In [6]:
# 直接测试 outline_analyzer.py
outline_analyzer = DocxOutlineAnalyzerStep()


In [7]:
input_datas = outline_analyzer.prepare_requests_data(docx_elements)

In [ ]:
print(type(input_datas))
print(input_datas[0])
print(input_datas[1])


In [ ]:
simulated_prompt, formatted_messages = outline_analyzer.simulate_prompt(input_datas[0])
pprint(formatted_messages)


In [ ]:
analysis_result = outline_analyzer.process(docx_elements)

In [ ]:
# 打印输出的结果
print(type(analysis_result))
print(type(analysis_result.analysis_result))
print(type(analysis_result.analysis_result.result))
print(type(analysis_result.analysis_result.result[0]))
pprint(analysis_result)


In [ ]:
saved_analysis = DocumentAnalysis.objects.get(title="测试分析A")
pprint(saved_analysis.outline_analysis_result)


# <font color="red">STEP 3: 测试outline_improvement.py</font>


In [ ]:
# 初始化
import django_setup

In [2]:
# 导入相关模型
from django.contrib.auth import get_user_model
from apps.doc_analysis.models import DocumentAnalysis, InvalidStatusTransition
from apps.projects.models import Project
from apps.files.models import FileRecord
from django.core.files.uploadedfile import SimpleUploadedFile
from django.core.exceptions import ValidationError
from apps.doc_analysis.steps._01_extract_docx_elements import DocxExtractorStep
from apps.doc_analysis.pipeline.types import ModelData, DocxElements, OutlineAnalysisResult
from apps.doc_analysis.steps._02_outline_analysis import DocxOutlineAnalyzerStep
from apps.doc_analysis.steps._03_outline_improvement import OutlineImprovementStep
from pprint import pprint 

In [3]:
# 直接引用"测试分析A", 并获取其docx_elements 和 outline_analysis_result

saved_analysis = DocumentAnalysis.objects.get(title="测试分析A")
docx_elements = DocxElements.from_model(saved_analysis.extracted_elements)
outline_analysis_result = OutlineAnalysisResult.from_model(saved_analysis.outline_analysis_result)

In [ ]:
# 打印输出的结果
print(type(outline_analysis_result.analysis_result))
print(type(outline_analysis_result.analysis_result.result))
print(type(outline_analysis_result.analysis_result.result[0]))
pprint(outline_analysis_result)


## 分步骤测试

In [5]:
# 构建实例
outline_improver = OutlineImprovementStep()

In [ ]:
# titles 提取
titles_to_improve = outline_improver.extract_titles_from_analysis(outline_analysis_result)
pprint(titles_to_improve)


In [8]:
# 改进元素
improved_elements = outline_improver.improve_document_elements(docx_elements, titles_to_improve)

In [ ]:
# 打印改进后的数据
pprint(improved_elements)

## 集成测试

In [ ]:
# 模拟用户确认
outline_analysis_result.user_confirm = True

pprint(outline_analysis_result)


In [6]:
# 完整测试 _02_outline_analysis.py
outline_improver = OutlineImprovementStep()
improved_docx_elements = outline_improver.process((docx_elements, outline_analysis_result))

In [ ]:
# 打印原来的数据
pprint(docx_elements)

In [ ]:
# 打印改进后的数据
pprint(improved_docx_elements)

# <font color=red>STEP 4: 构建文档树DocxTree <font>


In [ ]:
# 初始化
import django_setup

In [2]:
# show_structure函数（）
from pprint import pprint
from typing import Any

def show_structure(data: Any, name: str = "data", max_depth: int = None) -> None:
    """
    显示数据的类型和结构的辅助函数
    
    参数:
        data: 要检查的数据
        name: 数据的名称/标签
        max_depth: pprint的最大深度
    """
    print(f"\n{'='*50}")
    print(f"检查对象: {name}")
    print(f"类型: {type(data)}")
    
    if hasattr(data, '__len__'):
        print(f"长度: {len(data)}")
    
    print("\n结构:")
    pprint(data, depth=max_depth, width=100)
    print(f"{'='*50}\n")


In [3]:
# 导入相关模型
from apps.doc_analysis.models import DocumentAnalysis
from apps.doc_analysis.pipeline.types import ModelData, ImprovedDocxElements, SimpleDocxNode, DocxTree
from apps.doc_analysis.steps._04_build_docxtree import BuildDocxTree
from pprint import pprint 

In [4]:
# 直接引用"测试分析A", 并获取 improved_docx_elements
saved_analysis = DocumentAnalysis.objects.get(title="测试分析A")
improved_docx_elements = ImprovedDocxElements.from_model(saved_analysis.improved_docx_elements)
#elements = improved_docx_elements.elements
#show_structure(elements)


## 构建文档树

In [ ]:
# 构建文档树
build_docx_tree = BuildDocxTree()
docx_tree = build_docx_tree.process(improved_docx_elements)
show_structure(docx_tree)


In [ ]:
# 按顺序打印所有节点
pprint(docx_tree._ordered_nodes)

In [ ]:
# 按树结构打印所有节点
docx_tree.print_tree()

In [ ]:
print(docx_tree.format_titles())

In [ ]:
# 检查文档树在models.py里的存储情况
saved_analysis = DocumentAnalysis.objects.get(title="测试分析A")
print(type(saved_analysis.docxtree))
pprint(saved_analysis.docxtree)


## 测试从数据库提取文档树

In [ ]:
# 还原：从models.py存储到doc_tree类型的还原
docx_tree_from_model = DocxTree.from_model(saved_analysis.docxtree)
print(type(docx_tree_from_model))
pprint(docx_tree_from_model)

In [ ]:
# 还原 ： 按顺序打印所有节点
pprint(docx_tree_from_model._ordered_nodes)

In [ ]:
# 还原： 按树结构打印所有节点
docx_tree_from_model.print_tree()

In [ ]:
print(docx_tree_from_model.format_titles())

## 测试文档树方法：查找节点，打印树，标题格式化

In [ ]:
# 查找某个节点 find_node()， 测试 .find_node() 方法，比如node_id = 78
node = docx_tree.find_node(78)
pprint(node)

In [ ]:
# 打印某个节点以下的文档树结构
docx_tree.print_tree(124)


In [ ]:
# 格式化 标题结构 用于大模型分析
titles_nodes = docx_tree.format_titles()
pprint(titles_nodes)

## 测试添加节点

In [ ]:
# 添加 节点
docx_tree.add_title_node(
    content = '>>>> 插入新节点',
    level = 2,
    after_node_id = 150 
)

In [ ]:
# 打印结果 - 看顺序列表里是否有插入？
pprint(docx_tree._ordered_nodes[140:160])

In [ ]:
# 打印文档树
docx_tree.print_tree(124)

## 测试调整节点类型

In [19]:
# 节点改变层级
docx_tree.convert_to_title_node(156,2)

In [ ]:
# 打印结果 - 看顺序列表里是否有插入？
pprint(docx_tree._ordered_nodes[140:160])

In [ ]:
# 打印文档树
docx_tree.print_tree(124)

# <font color=red>  STEP 5: 测试文档树的使用</font>

In [ ]:
# 初始化
import django_setup

In [2]:
# 导入相关模型
from apps.doc_analysis.models import DocumentAnalysis
from apps.doc_analysis.pipeline.types import ModelData,DocxTree, DocxTreeMoreTitles
from apps.doc_analysis.steps._04_build_docxtree import BuildDocxTree
from apps.doc_analysis.steps._05_more_subtitles import MoreSubTitlesStep
from pprint import pprint 

In [ ]:
# 直接引用"测试分析A", 并获取 doc_tree
saved_analysis = DocumentAnalysis.objects.get(title="测试分析A")
docx_tree = DocxTree.from_model(saved_analysis.docxtree)
print(type(docx_tree))
print(docx_tree.format_titles())

In [ ]:
leaf_titles = docx_tree.get_leaf_titles()
pprint(leaf_titles)

In [ ]:
# 创建子标题分析 实例
subtitles_analyzer = MoreSubTitlesStep()
# 检查 data_inputs
data_inputs = subtitles_analyzer.prepare_requests_data(docx_tree)
# 检查prompt
simulated_prompt, formatted_messages = subtitles_analyzer.simulate_prompt(data_inputs[0])
print(formatted_messages)

In [ ]:
analysis_reuslts = subtitles_analyzer.process(docx_tree)

In [ ]:
print(type(analysis_reuslts))
pprint(analysis_reuslts)

In [ ]:
# 检查 more_subtitles 在models.py里的存储情况
saved_analysis = DocumentAnalysis.objects.get(title="测试分析A")
print(type(saved_analysis.more_subtitles))
pprint(saved_analysis.more_subtitles)

In [13]:
# 测试 从models.py提取 more_subtitles, 并转换为DocxTreeMoreTitles格式
more_subtitles_from_model = DocxTreeMoreTitles.from_model(saved_analysis.more_subtitles)
print(type(more_subtitles_from_model))
pprint(more_subtitles_from_model)